<a href="https://colab.research.google.com/github/SAQUIBZEYA/ML_Project/blob/master/LSTM_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading and plotting a data

In [0]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


data=pd.read_csv("/content/international-airline-passengers.csv")

In [8]:
data.head()

,Month,passengers_in_100_thousands
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [9]:
data.tail()

,Month,passengers_in_100_thousands
139,1960-08,606
140,1960-09,508
141,1960-10,461
142,1960-11,390
143,1960-12,432


Building the LSTM Model

In [0]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

Data prepration

In [17]:
ts_data=data.iloc[:,1]

# We apply the MinMax scaler from sklearn
# to normalize data in the (0, 1) interval.

scaler=MinMaxScaler(feature_range=(0,1))
dataset=scaler.fit_transform(pd.DataFrame(ts_data))
dataset[0:5]

array([[0.01544402],
       [0.02702703],
       [0.05405405],
       [0.04826255],
       [0.03281853]])

split the data into training and test

In [0]:
TRAIN_SIZE=0.70
train_size=int(len(dataset)* TRAIN_SIZE)
test_size=len(dataset)-train_size
train,test=dataset[0:train_size, :],dataset[train_size:len(dataset), :]
                                  

Get data into shape to use in keras

In [0]:
def create_dataset(dataset, window_size = 1):
    data_X, data_Y = [], []
    for i in range(len(dataset) - window_size - 1):
        a = dataset[i:(i + window_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + window_size, 0])
    return(np.array(data_X), np.array(data_Y))

In [21]:
# Create test and training sets for one-step-ahead regression.

window_size=10

train_X,train_Y= create_dataset(train,window_size)
test_X,test_Y =create_dataset(test,window_size)
print("original training data shape:")
print(train_X.shape)

original training data shape:
(89, 10)


In [22]:
# Reshape the input data into appropriate form for Keras.

train_X=np.reshape(train_X,(train_X.shape[0],1, train_X.shape[1]))
test_X=np.reshape(test_X,(test_X.shape[0],1, test_X.shape[1]))
print("New Training data shape:")
print(train_X.shape)

New Training data shape:
(89, 1, 10)


Build simple LSTM model on training data

In [23]:
def fit_model(train_X,train_Y,window_size=10):
  model=Sequential()
  model.add(LSTM(4,input_shape=(1,window_size)))
  model.add(Dense(1))
  model.compile(loss="mean_squared_error",optimizer='adam')
  model.fit(train_X,train_Y,epochs=100,batch_size=1,verbose=2)
  return(model)
# Fit the first model.
model1 = fit_model(train_X, train_Y, window_size)

Epoch 1/100
 - 1s - loss: 0.0336
Epoch 2/100
 - 0s - loss: 0.0087
Epoch 3/100
 - 0s - loss: 0.0061
Epoch 4/100
 - 0s - loss: 0.0052
Epoch 5/100
 - 0s - loss: 0.0047
Epoch 6/100
 - 0s - loss: 0.0042
Epoch 7/100
 - 0s - loss: 0.0041
Epoch 8/100
 - 0s - loss: 0.0038
Epoch 9/100
 - 0s - loss: 0.0034
Epoch 10/100
 - 0s - loss: 0.0034
Epoch 11/100
 - 0s - loss: 0.0033
Epoch 12/100
 - 0s - loss: 0.0031
Epoch 13/100
 - 0s - loss: 0.0030
Epoch 14/100
 - 0s - loss: 0.0029
Epoch 15/100
 - 0s - loss: 0.0029
Epoch 16/100
 - 0s - loss: 0.0028
Epoch 17/100
 - 0s - loss: 0.0026
Epoch 18/100
 - 0s - loss: 0.0026
Epoch 19/100
 - 0s - loss: 0.0025
Epoch 20/100
 - 0s - loss: 0.0024
Epoch 21/100
 - 0s - loss: 0.0022
Epoch 22/100
 - 0s - loss: 0.0023
Epoch 23/100
 - 0s - loss: 0.0023
Epoch 24/100
 - 0s - loss: 0.0023
Epoch 25/100
 - 0s - loss: 0.0022
Epoch 26/100
 - 0s - loss: 0.0022
Epoch 27/100
 - 0s - loss: 0.0022
Epoch 28/100
 - 0s - loss: 0.0021
Epoch 29/100
 - 0s - loss: 0.0021
Epoch 30/100
 - 0s - lo

Prediuction and model evaluation

In [24]:
def predict_and_score(model, X, Y):
    # Make predictions on the original scale of the data.
    pred = scaler.inverse_transform(model.predict(X))
    # Prepare Y data to also be on the original scale for interpretability.
    orig_data = scaler.inverse_transform([Y])
    # Calculate RMSE.
    score = math.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))
    return(score, pred)

rmse_train, train_predict = predict_and_score(model1, train_X, train_Y)
rmse_test, test_predict = predict_and_score(model1, test_X, test_Y)

print("Training data score: %.2f RMSE" % rmse_train)
print("Test data score: %.2f RMSE" % rmse_test)

Training data score: 20.11 RMSE
Test data score: 38.86 RMSE
